In [60]:
import torch
import torch.nn as nn
import jieba
from torch.utils.data import DataLoader


class Module(nn.Module):
    def __init__(self, word_count):
        super(Module, self).__init__()
        self.embed = nn.Embedding(num_embeddings=word_count, embedding_dim=256)
        self.rnn = nn.RNN(input_size=256, hidden_size=256, num_layers=1)
        self.linear = nn.Linear(in_features=256, out_features=word_count)

    def forward(self, input):
        embed = self.embed(input)
        output, hidden = self.rnn(embed)
        output = self.linear(output)
        return output


model = Module(word_count=10000)
input = torch.tensor([[1, 2]])
print(input.shape)
output = model(input)
print(output)

torch.Size([1, 2])
tensor([[[-0.1452,  0.1815,  0.1631,  ..., -0.3848,  0.0853,  0.1762],
         [-0.2401,  0.3554, -0.2260,  ...,  0.1055, -0.6106,  0.0722]]],
       grad_fn=<ViewBackward0>)
